In [1]:
import gensim
#import logging

#import string
from nltk.tokenize import RegexpTokenizer # tokenizing
from nltk.corpus import stopwords  # list of stop words
from nltk.stem.wordnet import WordNetLemmatizer # lemmatizer

from itertools import product
import numpy as np

from bs4 import BeautifulSoup
import requests

import time
import pandas as pd

import html

from collections import defaultdict
# Logging code taken from http://rare-technologies.com/word2vec-tutorial/
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Loading model

Choose one option from below.  The first 2 are the same model, which can be used to calculate a probability score for a phrase. The first is just the word vectors (no hidden layer weights). The second is the full model (can calculate score).

The last file is the negative sampling model, which always for the 'predict_output_word' function

In [2]:
# load full model with softmax, CBOW, and no negative sampling
path = "/Users/stevenfelix/Documents/DataScience_local/Insight/"
file = 'model_full_50M_sg0_sz250_win5_min3_hs1_neg0'
model= gensim.models.word2vec.Word2Vec.load(path+file)

# Prediction algorithm

In [ ]:
"""take a single phrase as a string, and preprocess, return tokens"""
tokenizer = RegexpTokenizer(r'\w+') # tokens separated by white spice
stops = set(stopwords.words('english')) # list of english stop words
lemma = WordNetLemmatizer()

def clean(title, rmv_stop_words=False):
    tokens = tokenizer.tokenize(title.lower())     # tokenize
    if rmv_stop_words:
        tokens = [i for i in tokens if not i in stops] # remove stop words
    normalized = [lemma.lemmatize(token) for token in tokens] # lemma
    return normalized

In [45]:
import re

contractions = re.compile(r"'|-|\"")
# all non alphanumeric
symbols = re.compile(r'(\W+)', re.U)
# single character removal
singles = re.compile(r'(\s\S\s)', re.I|re.U)
# separators (any whitespace)
seps = re.compile(r'\s+')
# tokenizer
tokenizer = RegexpTokenizer(r'\w+') # tokens separated by white spice
# stop words
stops = set(stopwords.words('english')) # list of english stop words

# cleaner (order matters)
def clean(text, rmv_stop_words=True, return_tokens=False): 
    text = text.lower()
    text = contractions.sub('', text)
    text = symbols.sub(r' \1 ', text)
    text = singles.sub(' ', text)
    text = seps.sub(' ', text)
    tokens = tokenizer.tokenize(text)     # tokenize
    if rmv_stop_words:
        tokens = [i for i in tokens if not i in stops] # remove stop words
        text = ' '.join(tokens)
    if return_tokens:
        return tokens
    return text

# sentence splitter
#alteos = re.compile(r'([!\?])')
#def sentences(l):
#    l = alteos.sub(r' \1 .', l).rstrip("(\.)*\n")
#    return l.split(".")

In [46]:
""" These generate alternative queries and score them and filter them """
def generate_alternatives(query, n, model, rmv_stop_words=True, return_tokens=True):
    print('getting similar words')
    syns = get_similar(query, n, model, rmv_stop_words=rmv_stop_words, return_tokens=return_tokens) # synonyms
    print('making combinations')
    combs = get_combinations(syns) # combinations
    # generatings probaiblity scores
    probs = [model.score([sug])[0] for sug in combs] # probabilities
    preds_probs =[(p,q) for p,q in zip(probs,combs)] # combine with queries
    q_score = model.score([tokenizer.tokenize(query)])[0] # score for original query
    sd = get_sd(preds_probs)
    preds_1sd = [(x,y) for x,y in preds_probs if np.abs(x-q_score)<=sd] # keep just those within 1 sd
    preds_1sd.sort(reverse=True)
    #print("original query: {}".format(query))
    #print("score: {}".format(q_score))
    #print("sd of all results: {}".format(sd))
    #print("number of results within 1 SD of original query score: {}".format(len(preds_1sd)))
    return preds_1sd

def get_similar(query, n, model, rmv_stop_words, return_tokens):
    q = clean(query, rmv_stop_words=rmv_stop_words, return_tokens=return_tokens)
    # turn each word  of query into its own list
    d = [[x] for x in q]
    for x in d:
        # for each word in original query, add topn similar words to list
        x.extend([syn for syn,_ in model.most_similar(x[0],topn=n)])
    return d

def get_combinations(l):
    combs = [x for x in product(*l)]
    return combs

def get_sd(tups):
    vals = [x for x,_ in tups]
    return np.std(vals)

def clean_preds(pred_scores, topn=3):
    clean = []
    i = 0
    for score,query in pred_scores:
        i+=1
        if i > topn: break
        clean.append((score, ' '.join(query)))
    return clean

# Validation algorithms

In [99]:
"""These query stack overflow and return and parse the serach results for validation"""

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
            'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            'Accept_Encoding': 'gzip, deflate, sdch, br', 'Accept_Language':'en-US,en;q=0.8',
            'Connection': 'keep-alive'}

def get_query_results(query):
    url = 'https://stackoverflow.com/search?q='+'+'.join(query)
    #print(url)
    r = requests.get(url)
    #print(r.status_code)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        total_results = soup.find("div", class_="subheader results-header").find('h2').get_text()
    except AttributeError:
        print("Stack overflow doesn't bleive you're human. Click the link and refresh to verify:")
        print(url)
        input("Press Enter to continue...")
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'lxml')
        total_results = soup.find("div", class_="subheader results-header").find('h2').get_text()
    x = parse_results(soup, total_results, url)
    print('returned {} of {} results'.format(len(x['results_summary']), x['total_results']))
    return x
    
def parse_results(soup, total, url):
    results_dict = {}
    l =[]
    results_dict['total_results'] = int(re.match('\d*',total).group())
    results = soup.find_all("div", class_="question-summary search-result")
    i = 0
    for result in results:
        i += 1
        if i > 10:
            break
        votes = [v.get_text() for v in result.find_all("strong")]
        if len(votes) == 1:
            votes.append(1)
        link = result.find("div", class_="result-link").find('a')
        query = link.attrs['title']
        url = link.attrs['href']
        votes.extend([query,url])
        l.append(tuple(votes))
    results_dict['results_summary'] = l
    return results_dict

In [87]:
""" given dictionaries of suggested-queries' results and the original query results,
    compares metrics of the returned queries"""
def compare(suggestions, original):
    # get result titles from original in a list
    orig_q = list(original)[0]
    orig_items = [title for _,_,title,_ in original[orig_q]['results_summary']]
    summaries = defaultdict(dict)
    orig_summaries = defaultdict(dict)
    for k in suggestions:
        # isolate titles for suggested results
        sug_items = [title for _,_,title,_ in suggestions[k]['results_summary']]
        # identify which results are shared with the original search results
        shared = set(orig_items) & set(sug_items)
        orig_unique = list(set(orig_items).difference(set(sug_items)))
        sug_unique = list(set(sug_items).difference(set(orig_items)))
        if len(sug_items) > 0:
            concordance = len(shared)*1.0/len(sug_items)
        else:
            concordance = np.nan
        
        votes = 0
        answers = 0
        for vote,answer,query,_ in original[orig_q]['results_summary']:
            if query in shared:
                continue
            votes += int(vote)
            answers += int(answer)
        orig_summaries[k]['unique'] = len(orig_unique)
        try: 
            # among the unique search results for the original query, what's the avg
            orig_summaries[k]['avg_unique_votes'] = votes*1.0/orig_summaries[k]['unique']
            orig_summaries[k]['avg_unique_answers'] = answers*1.0/orig_summaries[k]['unique']
        except ZeroDivisionError:
            orig_summaries[k]['avg_unique_votes'] = 0
            orig_summaries[k]['avg_unique_answers'] = 0
        orig_summaries[k]['titles'] = orig_unique
        #orig_summaries[k]['concordance'] = concordance
        #orig_summaries[k]['total_results'] = original[orig_q]['total_results']
        # calculate sum of votes and answers for all unique search results in suggestion
        votes = 0
        answers = 0
        #distances = []
        for vote,answer,query,_ in suggestions[k]['results_summary']:
            if query in shared:
                continue
            votes += int(vote)
            answers += int(answer)
            #distances.append ()# cosine distance fucntion here)
            # concordance
        summaries[k]['unique'] = len(sug_unique)
        summaries[k]['titles'] = sug_unique
        summaries[k]['concordance'] = concordance
        summaries[k]['total_results_dif'] = suggestions[k]['total_results'] - original[orig_q]['total_results']
        try: 
            summaries[k]['avg_unique_votes'] = votes*1.0/summaries[k]['unique']
            summaries[k]['avg_unique_answers'] = answers*1.0/summaries[k]['unique']
        except ZeroDivisionError:
            summaries[k]['avg_unique_votes'] = 0
            summaries[k]['avg_unique_answers'] = 0
    return summaries, orig_summaries

"""
def make_summaries(key, dic):
    d = defaultdict(dict)
    votes = 0
    answers = 0
    for vote,answer,query,_ in dic[list(original)[0]]:
        if query in shared:
            continue
            votes += int(vote)
            answers += int(answer)
        d[key]['votes'] = votes
        d[key]['answers'] = answers
        d[key]['unique'] = orig_unique
        d[key]['unique_perc'] = len(orig_unique)*1.0/len(orig_items)
        """

"\ndef make_summaries(key, dic):\n    d = defaultdict(dict)\n    votes = 0\n    answers = 0\n    for vote,answer,query,_ in dic[list(original)[0]]:\n        if query in shared:\n            continue\n            votes += int(vote)\n            answers += int(answer)\n        d[key]['votes'] = votes\n        d[key]['answers'] = answers\n        d[key]['unique'] = orig_unique\n        d[key]['unique_perc'] = len(orig_unique)*1.0/len(orig_items)\n        "

In [100]:
def validate(test_queries, n_syns, topn, model, dic, rmv_stop_words=True, return_tokens=True):
    #results_dict = defaultdict(dict)
    i = 0
    for q in test_queries:
        i+=1
        print('starting query {}'.format(i))
        dic[q] = evaluate_query(q, n_syns, topn, model, rmv_stop_words=rmv_stop_words, return_tokens=return_tokens)
        if dic[q] == None:
            print('query {} failed'.format(i))
            continue
        print('query {} completed'.format(i))
    return dic
        #results_dict[q] = evaluate_query(q, n_syns, topn, model)
    #return results_dict

def evaluate_query(orig_query, n_syns, topn, model, rmv_stop_words, return_tokens):
    # improtant local varaibles
    orig_query_vec = clean(orig_query, rmv_stop_words=rmv_stop_words, return_tokens=return_tokens)

    # generate, rank, filter suggestions
    print('generating predictions')
    preds = generate_alternatives(orig_query, n_syns, model)
    best_preds = clean_preds(preds, topn)
    best_preds2 = [(s,q) for s,q in best_preds if q!=' '.join(orig_query_vec)]
    #orig = (model.score([orig_query_vec])[0], ' '.join(orig_query_vec))
    
    # query each suggestion and scrape relevent metrics
    sug_q_results ={}
    for _,q in best_preds2:
        print('getting search results for \"{}\"'.format(q))
        sug_q_results[q] = get_query_results(tokenizer.tokenize(q))
        #print(sug_q_results[q])
        #if sug_q_results[q] == []:
        #    print('Website returned no results.')
        #    return None
        time.sleep(4)
    
    # get query results for original query
    orig_q_results = {orig_query: get_query_results(orig_query_vec)}
    
    # get metrics
    print('making comparisons')
    summaries,orig_summaries = compare(sug_q_results, orig_q_results)
    
    # calculate average metrics
    avg_answers_dif_score = np.mean([summaries[k]['avg_unique_answers'] - 
                                  orig_summaries[k]['avg_unique_answers'] for k in summaries])
    avg_votes_dif_score = np.mean([summaries[k]['avg_unique_votes'] - 
                                orig_summaries[k]['avg_unique_votes'] for k in summaries])
    avg_unique_items = np.mean([summaries[k]['unique'] for k in summaries])
    avg_answers_dif_score = np.mean([summaries[k]['avg_unique_answers'] - 
                                     orig_summaries[k]['avg_unique_answers'] for k in summaries])
    
    avg_total_results_dif_score = np.mean([summaries[k]['total_results_dif'] for k in summaries])
    avg_concordance = np.mean([summaries[k]['concordance'] for k in summaries])
    
    results = {}
    orig_prob = model.score([orig_query_vec])[0]
    results['avg_unique_items'] = avg_unique_items
    results['avg_answers_dif_score'] = avg_answers_dif_score
    results['avg_votes_dif_score'] = avg_votes_dif_score
    results['avg_prob_dif_score'] = np.mean([np.exp(prob)-np.exp(orig_prob) for prob,q in best_preds2])/np.exp(orig_prob)
    results['avg_total_results_dif_score'] = avg_total_results_dif_score
    results['avg_concordance'] = avg_concordance
    return results

# Brief test

In [71]:
queries = ['initializing row pandas dataframe']

In [72]:
r = defaultdict(dict)
results = validate(queries, n_syns=5, topn=5, model=model, dic=r)

starting query 1
generating predictions
getting similar words
making combinations
getting search results for "initialization column pandas dataframe"
https://stackoverflow.com/search?q=initialization+column+pandas+dataframe
returned 10 of 189 results
getting search results for "initializing column pandas dataframe"
https://stackoverflow.com/search?q=initializing+column+pandas+dataframe
returned 10 of 188 results
getting search results for "initialization rows pandas dataframe"
https://stackoverflow.com/search?q=initialization+rows+pandas+dataframe
returned 10 of 111 results
getting search results for "initialization row pandas dataframe"
https://stackoverflow.com/search?q=initialization+row+pandas+dataframe
returned 10 of 111 results
getting search results for "initializing rows pandas dataframe"
https://stackoverflow.com/search?q=initializing+rows+pandas+dataframe
returned 10 of 111 results
https://stackoverflow.com/search?q=initializing+row+pandas+dataframe
returned 10 of 111 results

In [73]:
results

defaultdict(dict,
            {'initializing row pandas dataframe': {'avg_answers_dif_score': 0.14285714285714285,
              'avg_concordance': 0.61999999999999988,
              'avg_prob_dif_score': 3.462512,
              'avg_total_results_dif_score': 31.0,
              'avg_unique_items': 3.7999999999999998,
              'avg_votes_dif_score': -1.3535714285714286}})

In [74]:
pd.DataFrame(results).T

,avg_answers_dif_score,avg_concordance,avg_prob_dif_score,avg_total_results_dif_score,avg_unique_items,avg_votes_dif_score
initializing row pandas dataframe,0.142857,0.62,3.462512,31.0,3.8,-1.353571


# Getting queries for validation

In [ ]:
from stackapi import StackAPI
from datetime import datetime, timedelta

In [ ]:
# create connection
SITE = StackAPI('stackoverflow')
SITE.page_size = 100
SITE.max_pages = 6

In [ ]:
def get_attrib(posts, tag):
    return [d[tag] for d in posts['items']]

def append(filename, dat):    
    with open(filename, 'a+') as f:
        for item in dat:
            f.write(item+'\n')

In [ ]:
posts = SITE.fetch('questions', fromdate=datetime(2010,1,1), todate=datetime(2016,1,1),
                                min=10)

In [ ]:
print('Available Attributes: ')
list(posts['items'][0])

In [ ]:
def get_attrib(attrib):
    return [x[attrib] for x in posts['items']]

In [ ]:
np.mean(get_attrib('score'))

In [ ]:
[datetime.fromtimestamp(date) for date in get_attrib('creation_date')]

 - dates are randomly scattered between 2010 and 2015

In [ ]:
import json

with open(path+'posts_for_validation.txt', 'w') as outfile:
    json.dump(posts, outfile)

with open(path+'titles_for_validation.txt', 'w') as f:
    for title in get_attrib('title'):
        f.write(title+'\n')

### no need to use these

In [ ]:
def get_posts(iters, connection, typ, path,fname):
    for i in range(iters):
        posts = connection.fetch(typ, fromdate=datetime(2010,1,1), todate=datetime(2015,1,1),
                                min=10, sort='votes')
    
        # parse and write titles
        titles = get_attrib(posts, 'title')
        append(path+fname, titles)

        print('waiting {} seconds'.format(60+posts['backoff']))
        time.sleep(60+posts['backoff'])

In [ ]:
path = '/Users/stevenfelix/Documents/DataScience_local/Insight/'
get_posts(1, SITE, 'questions', path, 'recent_titles.txt')

# Validation

In [75]:
with open(path+'titles_for_validation.txt', 'r') as f:
    queries = f.readlines()

queries =[html.unescape(q).strip() for q in queries]

In [76]:
queries100 = queries[0:100]
len(queries100)
## note, lots of special characters will get stripped in my pre-processor, 
## meaning that the original query will almost always perform better
## need to retain proper names and commong things (C++, robots.txt, C#, function())

100

In [79]:
for q in queries100:
    print(q)
    tok = clean(q, rmv_stop_words=True, return_tokens=True)
    print(tok)
    print('\n')
# do remove stop words
# don't remove trailing s !!

Install RoR on debian Squeeze
['install', 'ror', 'debian', 'squeeze']


CSS Line Height - bottom only
['css', 'line', 'height', 'bottom']


Writing string to a file on a new line every time
['writing', 'string', 'file', 'new', 'line', 'every', 'time']


mysql error: Can't create a new thread (errno 11)
['mysql', 'error', 'cant', 'create', 'new', 'thread', 'errno', '11']


CSS line-height alignment issues
['css', 'lineheight', 'alignment', 'issues']


Return 410 for all but robots.txt
['return', '410', 'robots', 'txt']


Swift: Change a button color once pressed
['swift', 'change', 'button', 'color', 'pressed']


Linked server can't start distributed transaction in trigger
['linked', 'server', 'cant', 'start', 'distributed', 'transaction', 'trigger']


lodash : how to loop with between a start value and end value
['lodash', 'loop', 'start', 'value', 'end', 'value']


Category implementation stuck in thread with increasing memory heap space
['category', 'implementation', 'stuck', 'thread

In [81]:
queries_clean_7 = []
for q in queries100:
    tok = clean(q, rmv_stop_words=True, return_tokens=True)
    if len(tok) > 7:
        continue
    queries_clean_7.append(' '.join(tok))

In [82]:
print(len(queries_clean_7))
queries_clean_7[0:10]

82


['install ror debian squeeze',
 'css line height bottom',
 'writing string file new line every time',
 'css lineheight alignment issues',
 'return 410 robots txt',
 'swift change button color pressed',
 'linked server cant start distributed transaction trigger',
 'lodash loop start value end value',
 'unable initialize mockhttpservletrequest unit tests',
 'add border top side uiview']

In [ ]:
#r = defaultdict(dict)
results = validate(queries_clean_7, n_syns=3, topn=5, model=model, dic=r)

In [146]:
queries_clean_7_mod = [q for q in queries_clean_7 if q not in list(r)]
print(len(queries_clean_7_mod))

4


In [144]:
queries_clean_7_mod = queries_clean_7_mod[4:]

In [145]:
results = validate(queries_clean_7_mod, n_syns=3, topn=5, model=model, dic=r)

starting query 1
generating predictions
getting similar words
making combinations
getting search results for "sklearn countvectorizer variable multiselect"
Stack overflow doesn't bleive you're human. Click the link and refresh to verify:
https://stackoverflow.com/search?q=sklearn+countvectorizer+variable+multiselect
Press Enter to continue...
returned 0 of 0 results
getting search results for "sklearn countvectorizer variables selected"
returned 0 of 0 results
getting search results for "sklearn countvectorizer variables multiselect"
returned 0 of 0 results
getting search results for "scikitlearn countvectorizer variable multiselect"
returned 0 of 0 results
getting search results for "scikitlearn countvectorizer variables multiselect"
returned 0 of 0 results
returned 1 of 1 results
making comparisons
query 1 completed
starting query 2
generating predictions
getting similar words
making combinations
getting search results for "tinymce open jquery modal dialog"
returned 10 of 23 results


/Users/stevenfelix/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/stevenfelix/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


making combinations
getting search results for "rails json object model association"
returned 10 of 61 results
getting search results for "rails json objects model associations"
returned 10 of 61 results
getting search results for "rails json object models associations"
returned 10 of 62 results
getting search results for "rails json objects model association"
returned 10 of 61 results
returned 10 of 61 results
making comparisons
query 7 completed
starting query 8
generating predictions
getting similar words
making combinations
getting search results for "correct proguard configuration keep static inner classes"
returned 2 of 2 results
getting search results for "correct proguard configuration keep static inner class"
returned 2 of 2 results
getting search results for "correct proguard configuration keep private inner classes"
returned 1 of 1 results
getting search results for "correct proguard configuration keep private inner class"
returned 1 of 1 results
getting search results for "

In [119]:
pd.DataFrame(r).T

,avg_answers_dif_score,avg_concordance,avg_prob_dif_score,avg_total_results_dif_score,avg_unique_items,avg_votes_dif_score
access table comment mysql table,1.256250e+00,0.075000,0.050452,-206.00,9.25,14.613889
add border top side uiview,-2.715556e+00,0.020000,12.897857,-7.40,9.40,-2.217778
android achartengine points yaxis,7.500000e-01,NaN,-0.326893,2.50,2.50,0.500000
android options menu fragment,-1.444444e+00,0.200000,1.605973,-281.50,7.75,-30.872222
browsers still support css expressions,-5.316667e-01,NaN,1.813610,18.20,4.00,77.231667
c redirect stream console,-4.800000e-01,0.460000,6.649093,9.00,5.40,99.100000
calculate age years months days,-4.100000e-01,0.320000,0.732957,16.60,6.80,-0.020000
cant align two html5 input range,0.000000e+00,NaN,11.290619,0.00,0.00,0.000000
change default message permissions location manager ios,-1.000000e+00,NaN,1.907806,-2.00,0.00,-8.250000
com sun jdi invocationexception occurred invoking method,-2.500000e-01,NaN,-0.754492,-0.25,0.00,0.000000


In [132]:
r2 = dict(r)
r2.pop(0)

['install ror debian squeeze',
 'css line height bottom',
 'writing string file new line every time',
 'css lineheight alignment issues',
 'return 410 robots txt',
 'swift change button color pressed',
 'linked server cant start distributed transaction trigger',
 'lodash loop start value end value',
 'unable initialize mockhttpservletrequest unit tests',
 'add border top side uiview',
 'android options menu fragment',
 'get white circular progress bar android',
 'change default message permissions location manager ios',
 'error creating runnable jar eclipse',
 'android achartengine points yaxis',
 'sql server stops loading assembly',
 'create dynamic directory structure sd card android',
 'sheets displaying properly osx',
 'seckeychainfindgenericpassword working osx',
 'scan available bluetooth devices range android',
 'convert json object duplicate keys json array',
 'whats good hash function english words',
 'tint menu icons',
 'order calling constructors python',
 'access table comm

In [147]:
import json

with open(path+'validation_results.txt', 'w') as outfile:
    json.dump(r2, outfile)

TypeError: Object of type 'float32' is not JSON serializable

In [148]:
results_df = pd.DataFrame(r).T
results_df.head()

,avg_answers_dif_score,avg_concordance,avg_prob_dif_score,avg_total_results_dif_score,avg_unique_items,avg_votes_dif_score
install ror debian squeeze,-1.172222,0.240,10.434649,10.00,6.8,0.227778
css line height bottom,-3.394841,0.100,-0.773102,-0.75,9.0,-154.059127
writing string file new line every time,-0.300694,0.075,0.138001,70.75,7.5,325.987500
css lineheight alignment issues,1.340000,NaN,3.757933,77.80,6.2,5.760000
return 410 robots txt,0.700000,NaN,166.547943,3.40,3.0,5.960000


In [149]:
results_df.to_csv(path+'validation_results_df.csv')

In [150]:
results_df.mean()

avg_answers_dif_score          -0.730413
avg_concordance                 0.264853
avg_prob_dif_score             29.145062
avg_total_results_dif_score    -6.198052
avg_unique_items                3.881169
avg_votes_dif_score            20.338726
dtype: float64

6.0

suggestions:
 - have .73 LESS answers per result
 - lead to results with 26% overlap
 - are 29 times more probable
 - retrieve 6.19 fewer results
 - have ~4 unique items
 - have 20 more votes
 
Note: this method is skewed negatively -- the "original query" was derived from a real post, so it almost always will get a result, whereas the suggestions may not (sot he comparison will always be in favor of original)

- need to preprocess in a way that either keeps periods 'robots.txt' or always separates them
- algorithm also needs to exclude substitutsions that are terrible (ie lead to 0 results)
- sometimes removing words would be highly beneficial

returned 0 of 0 results
getting search results for "sklearn countvectorizer variables selected"
returned 0 of 0 results
getting search results for "sklearn countvectorizer variables multiselect"
returned 0 of 0 results
getting search results for "scikitlearn countvectorizer variable multiselect"
returned 0 of 0 results
getting search results for "scikitlearn countvectorizer variables multiselect"
returned 0 of 0 results

getting search results for "apache mina writing iosession"
returned 1 of 1 results
getting search results for "apache mina writing messagerecieved"
returned 0 of 0 results
getting search results for "apache mina writing executorfilter"
returned 0 of 0 results
getting search results for "apache mina read iosession"

getting search results for "know nodetool repair completed"
returned 6 of 6 results
getting search results for "tell nodetool repair completed"
returned 0 of 0 results
getting search results for "tell nodetool repair completes"
returned 0 of 0 results
getting search results for "determine nodetool repair completed"
returned 0 of 0 results
getting search results for "tell nodetool repair finished"
returned 1 of 1 results